In [1]:
# # Tabnet
# !pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

In [2]:
!pip uninstall -y typing
!pip install ../input/pytorchtabnetpretraining/pytorch_tabnet-2.0.1-py3-none-any.whl
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
Processing /kaggle/input/pytorchtabnetpretraining/pytorch_tabnet-2.0.1-py3-none-any.whl
Processing /kaggle/input/iterative-stratification/iterative-stratification-master
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.6-py3-none-any.whl size=8401 sha256=8fb8aab5ebcfb4779625a343d044daf767548d38c58407ca2ab8960eb1bb22c0
  Stored in directory: /root/.cache/pip/wheels/b8/47/3f/eb4af42d124f37d23d6f13a4c8bbc32c1d70140e6e1cecb4aa
Successfully built iterative-stratification


In [3]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import pickle
import seaborn as sns
sns.set()

from sklearn import preprocessing
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [4]:
os.listdir('../input/lish-moa')

['sample_submission.csv',
 'train_drug.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'test_features.csv']

In [5]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [6]:
train_features.head(2)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371


In [7]:
train_targets_scored.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_targets_nonscored.head(2)

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test_features.head(2)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.143,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.151,-0.775
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.031,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.381,-0.730


In [10]:
sample_submission.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [11]:
seed = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASSED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed)

## preprocess data

In [12]:
def drop_ctl_vehicle(train_features, test_features):
    
    train_features = train_features[train_features['cp_type'] != 'ctl_vehicle']
    test_features = test_features[test_features['cp_type'] != 'ctl_vehicle']
    target = train_targets_scored.iloc[train_features.index]
    train_features.reset_index(drop=True, inplace=True)
    test_features.reset_index(drop=True, inplace=True)
    
    return train_features, test_features, target

# train_features, test_features, target = drop_ctl_vehicle(train_features, test_features)
# test_features

In [13]:
def rank_gauss1(train_features, test_features):
    
    train_features_ = train_features.copy()
    test_features_ = test_features.copy()
    
    GENES = [col for col in train_features_.columns if col.startswith('g-')]
    CELLS = [col for col in train_features_.columns if col.startswith('c-')]
    
    for col in (GENES + CELLS):

        transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
        vec_len = len(train_features_[col].values)
        vec_len_test = len(test_features_[col].values)
        raw_vec = train_features_[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)

        train_features_[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
        test_features_[col] = transformer.transform(test_features_[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
        
    return train_features_, test_features_

# train_features1, test_features1 = rank_gauss1(train_features, test_features)
# train_features1

In [14]:
def rank_gauss2(train_features, test_features):
    
    train_features_ = train_features.copy()
    test_features_ = test_features.copy()
    
    data = pd.concat([train_features_, test_features])
    
    GENES = [col for col in train_features_.columns if col.startswith('g-')]
    CELLS = [col for col in train_features_.columns if col.startswith('c-')]
    
    for col in (GENES + CELLS):

        transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
        vec_len = len(data[col].values)
#         vec_len_test = len(test_features_[col].values)
        raw_vec = data[col].values.reshape(vec_len, 1)
        raw_vec = transformer.fit_transform(raw_vec)

        train_features_[col] = raw_vec[:len(train_features)]
        test_features_[col] = raw_vec[len(train_features):]
        
    return train_features_, test_features_

# train_features1, test_features1 = rank_gauss2(train_features, test_features)
# train_features1

In [15]:
def get_stats(train_features, test_features):
    
    GENES = [col for col in train_features.columns if col.startswith('g-')]
    CELLS = [col for col in train_features.columns if col.startswith('c-')]
    
    for df in [train_features, test_features]:
#         df['g_sum'] = df[GENES].sum(axis = 1)
        df['g_mean'] = df[GENES].mean(axis = 1)
        df['g_std'] = df[GENES].std(axis = 1)
        df['g_kurt'] = df[GENES].kurtosis(axis = 1)
        df['g_skew'] = df[GENES].skew(axis = 1)
#         df['g_max'] = df[GENES].max(axis=1)
#         df['g_min'] = df[GENES].max(axis=1)
#         df['c_sum'] = df[CELLS].sum(axis=1)
        df['c_mean'] = df[CELLS].mean(axis = 1)
        df['c_std'] = df[CELLS].std(axis = 1)
        df['c_kurt'] = df[CELLS].kurtosis(axis = 1)
        df['c_skew'] = df[CELLS].skew(axis = 1)
#         df['c_max'] = df[CELLS].max(axis=1)
#         df['c_min'] = df[CELLS].min(axis=1)
#         df['gc_sum'] = df[GENES + CELLS].sum(axis = 1)
        df['gc_mean'] = df[GENES + CELLS].mean(axis = 1)
        df['gc_std'] = df[GENES + CELLS].std(axis = 1)
        df['gc_kurt'] = df[GENES + CELLS].kurtosis(axis = 1)
        df['gc_skew'] = df[GENES + CELLS].skew(axis = 1)
        
    return train_features, test_features

# train_features, test_features = get_stats(train_features, test_features)
# test_features

In [16]:
def get_pca1(train_features, test_features, n_gs, n_cs):
    
    GENES = [col for col in train_features.columns if col.startswith('g-')]
    CELLS = [col for col in train_features.columns if col.startswith('c-')]
    
    pca_gs = PCA(n_components = n_gs)
    pca_cs = PCA(n_components = n_cs)
    
    train_pca_gs = pca_gs.fit_transform(train_features[GENES])
    train_pca_cs = pca_cs.fit_transform(train_features[CELLS])
    test_pca_gs = pca_gs.transform(test_features[GENES])
    test_pca_cs = pca_cs.transform(test_features[CELLS])
    
    train_pca_gs = pd.DataFrame(train_pca_gs, columns=[f'pca_G-{i}' for i in range(n_gs)])
    train_pca_cs = pd.DataFrame(train_pca_cs, columns=[f'pca_C-{i}' for i in range(n_cs)])
    test_pca_gs = pd.DataFrame(test_pca_gs, columns=[f'pca_G-{i}' for i in range(n_gs)])
    test_pca_cs = pd.DataFrame(test_pca_cs, columns=[f'pca_C-{i}' for i in range(n_cs)])
    
    train_features = pd.concat([train_features, train_pca_gs, train_pca_cs], axis=1)
    test_features = pd.concat([test_features, test_pca_gs, test_pca_cs], axis=1)
    
    return train_features, test_features

# train_features1, test_features1 = get_pca1(train_features, test_features, n_gs=50, n_cs=15)
# test_features1

In [17]:
def get_pca2(train_features, test_features, n_gs, n_cs):
    
    GENES = [col for col in train_features.columns if col.startswith('g-')]
    CELLS = [col for col in train_features.columns if col.startswith('c-')]
    
    pca_gs = PCA(n_components = n_gs)
    pca_cs = PCA(n_components = n_cs)
    
    data = pd.concat([train_features, test_features])
    
    data_pca_gs = pca_gs.fit_transform(data[GENES])
    data_pca_cs = pca_cs.fit_transform(data[CELLS])
    
    train_pca_gs = data_pca_gs[:len(train_features), :]
    train_pca_cs = data_pca_cs[:len(train_features), :]
    test_pca_gs = data_pca_gs[len(train_features):, :]
    test_pca_cs = data_pca_cs[len(train_features):, :]
    
    train_pca_gs = pd.DataFrame(train_pca_gs, columns=[f'pca_G-{i}' for i in range(n_gs)])
    train_pca_cs = pd.DataFrame(train_pca_cs, columns=[f'pca_C-{i}' for i in range(n_cs)])
    test_pca_gs = pd.DataFrame(test_pca_gs, columns=[f'pca_G-{i}' for i in range(n_gs)])
    test_pca_cs = pd.DataFrame(test_pca_cs, columns=[f'pca_C-{i}' for i in range(n_cs)])
    
    train_features = pd.concat([train_features, train_pca_gs, train_pca_cs], axis=1)
    test_features = pd.concat([test_features, test_pca_gs, test_pca_cs], axis=1)
    
    return train_features, test_features

# train_features2, test_features2 = get_pca2(train_features, test_features, n_gs=50, n_cs=15)
# test_features2

In [18]:
def var_thresh(threshold, train_features, test_features):

    cols_numeric = [f for f in train_features.columns if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
    mask = (train_features[cols_numeric].var() >= 0.8).values
    train_features_ = train_features[cols_numeric].loc[:, mask]
    test_features_ = test_features[cols_numeric].loc[:, mask]
    train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], train_features_], axis=1).reset_index(drop=True)
    test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], test_features_], axis=1).reset_index(drop=True)

    return train_features, test_features

# train_features, test_features = var_thresh_2(threshold=0.2, train_features=train_features, test_features=test_features)
# test_features

In [19]:
def get_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 0):
    
    GENES = [col for col in train.columns if col.startswith('g-')]
    CELLS = [col for col in train.columns if col.startswith('c-')]
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        
        return train, test
    
    train, test = create_cluster(train, test, GENES, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, CELLS, kind = 'c', n_clusters = n_clusters_c)
    
    return train, test

# train_features, test_features = get_cluster(train_features, test_features, n_clusters_g=35, n_clusters_c=5, SEED=0)

In [20]:
def scaling(train_features, test_features):
    
    scaler = RobustScaler()
    train_features_ = train_features.drop(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], axis=1)
    columns_ = train_features_.columns
    train_features_numerical = scaler.fit_transform(train_features_)
    test_features_numerical = scaler.transform(test_features.drop(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], axis=1))
    train_features_ = pd.DataFrame(train_features_numerical)
    test_features_ = pd.DataFrame(test_features_numerical)
    train_features_ = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], train_features_], axis=1)
    test_features_ = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], test_features_], axis=1)
    
    return train_features_, test_features_

# train_features, test_features = scaling2(train_features, test_features)
# test_features

In [21]:
def make_folds(train, num_starts, num_splits):
    
    train_ = train.copy()
    folds = []

    # LOAD FILES
    train_feats = pd.read_csv('../input/lish-moa/train_features.csv')
    scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
    drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
    scored = scored.loc[train_feats['cp_type'] == 'trt_cp', :]
    drug = drug.loc[train_feats['cp_type'] == 'trt_cp', :]
    targets = scored.columns[1:]
    scored = scored.merge(drug, on='sig_id', how='left') 

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[(vc <= 6) | (vc == 12) | (vc == 18)].index.sort_values()
    vc2 = vc.loc[(vc > 6) & (vc != 12) & (vc != 18)].index.sort_values()

    for seed in range(num_starts):

        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}; dct2 = {}
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop = True)
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        scored['fold'] = scored.drug_id.map(dct1)
        scored.loc[scored.fold.isna(),'fold'] =\
            scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
        scored.fold = scored.fold.astype('int8')
        folds.append(scored.fold.values)

        del scored['fold']
        
        for i in range(len(folds)):
            train_[f'seed{i}'] = folds[i]

    return train_

In [22]:
def make_folds_old(train, n_splits):
    
    folds = train.copy()

    mskf = MultilabelStratifiedKFold(n_splits)

    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
        folds.loc[v_idx, 'kfold'] = int(f)

    folds['kfold'] = folds['kfold'].astype(int)
    
    return folds

In [23]:
def preprocessor(train_features, test_features):
    
    print('start')
    
#     # drop_ctl_vehicle
#     train_features, test_features, target = drop_ctl_vehicle(train_features, test_features)
#     print('drop_ctl_vehicle: done')
    
    # rank_gauss
    train_features, test_features = rank_gauss1(train_features, test_features)
    print('rank_gauss: done')
    print('train_features.shape: ', train_features.shape)
    print('test_features.shape: ', test_features.shape)
    
    # stats
    train_features, test_features = get_stats(train_features, test_features)
    print('get_stats: done')
    print('train_features.shape: ', train_features.shape)
    print('test_features.shape: ', test_features.shape)
    
    # pca
    train_features, test_features = get_pca1(train_features, test_features, n_gs=600, n_cs=50) 
    print('get_pca: done')
    print('train_features.shape: ', train_features.shape)
    print('test_features.shape: ', test_features.shape)
    
    # var_thresh
    train_features, test_features = var_thresh(threshold=0.8, train_features=train_features, test_features=test_features)
    print('var_thresh: done')
    print('train_features.shape: ', train_features.shape)
    print('test_features.shape: ', test_features.shape)
    
#     # clustering
#     train_features, test_features = get_cluster(train_features, test_features, n_clusters_g=35, n_clusters_c=5, SEED=0)
#     print('get_clustering: done')
#     print('train_features.shape: ', train_features.shape)
#     print('test_features.shape: ', test_features.shape)
    
#     # Scaling
#     train_features, test_features = scaling(train_features, test_features)
#     print('scaling: done')
    
    # data merge
    train = train_features.merge(train_targets_scored, on='sig_id')
    train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
    test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

    target = train[train_targets_scored.columns]
    
    train = train.drop('cp_type', axis=1)
    test = test.drop('cp_type', axis=1)
    
    return train, target, test  

train, target, test = preprocessor(train_features, test_features)

start
rank_gauss: done
train_features.shape:  (23814, 876)
test_features.shape:  (3982, 876)
get_stats: done
train_features.shape:  (23814, 888)
test_features.shape:  (3982, 888)
get_pca: done
train_features.shape:  (23814, 1538)
test_features.shape:  (3982, 1538)
var_thresh: done
train_features.shape:  (23814, 1044)
test_features.shape:  (3982, 1044)


In [24]:
folds = make_folds_old(train, n_splits=5)

In [25]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
feature_cols = [c for c in pd.get_dummies(train, columns=['cp_time', 'cp_dose']).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id']]
len(feature_cols)

1045

### shape check

In [26]:
print(f'train.shape: {train.shape}')
print(f'folds.shape: {folds.shape}')
print(f'test.shape: {test.shape}')
print(f'target.shape: {target.shape}')
print(f'sample_submission.shape: {sample_submission.shape}')

train.shape: (21948, 1249)
folds.shape: (21948, 1250)
test.shape: (3624, 1043)
target.shape: (21948, 207)
sample_submission.shape: (3982, 207)


In [27]:
all_data = {
    'train': train, 
    'folds': folds, 
    'test': test, 
    'target': target, 
    'sample_submission': sample_submission
}

In [28]:
with open('all_data_TabNet.pickle', 'wb') as f:
    pickle.dump(all_data, f)

## Dataset Classes

In [29]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float), 
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

### HyperParameters

In [30]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 35
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 5e-6
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False
smoothing = 1e-5
p_min = smoothing
p_max = 1 - smoothing

num_features = len(feature_cols)
num_targets = len(target_cols)
hidden_size_1 = 1024
hidden_size_2 = 1024

In [31]:
MAX_EPOCH = 200
# n_d and n_a are different from the original work, 32 instead of 24
# This is the first change in the code from the original
tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = optim.lr_scheduler.ReduceLROnPlateau,
    seed = seed,
    verbose = 1
)

### Model

### training function

In [32]:
def train_fn(model, optimizer, scheduler, loss_tr, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    final_metric = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_tr(outputs, targets)
        metric = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        final_metric += metric.item()
        
    final_loss /= len(dataloader)
    final_metric /= len(dataloader)
    
    return final_loss, final_metric

In [33]:
def valid_fn(model, loss_tr, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    final_metric = 0
    valid_preds = []
    
    for data in dataloader:
        
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_tr(outputs, targets)
        metric = loss_fn(outputs, targets)
        final_loss += loss.item()
        final_metric += metric.item()
        outputs = torch.clamp(torch.sigmoid(outputs).detach().cpu(), p_min, p_max)
        valid_preds.append(outputs.numpy())
        
    final_loss /= len(dataloader)
    final_metric /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
        
    return final_loss, final_metric, valid_preds

In [34]:
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)
        
        with torch.no_grad():
            outputs = model(inputs)
            
        outputs = torch.clamp(torch.sigmoid(outputs).detach().cpu(), p_min, p_max)    
        preds.append(outputs.numpy())
        
    preds = np.concatenate(preds)
        
    return preds

### loss_function

In [35]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(DEVICE) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss

In [36]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets
    
    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [37]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        
        logits = 1 / (1 + np.exp(-y_pred))
#         logits = np.clip(logits, p_min, p_max)
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

### Single fold training

In [38]:
# def run_training(fold, seed, EPOCHS, LEARNING_RATE):
    
#     print(f'==========FOLD{fold+1}==========')
    
    
#     seed_everything(seed)
    
#     train_losses = list()
#     valid_losses = list()
    
#     train = pd.get_dummies(folds, columns=['cp_time', 'cp_dose'])
#     test_ = pd.get_dummies(test, columns=['cp_time', 'cp_dose'])
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index()
#     valid_df = train[train['kfold'] == fold].reset_index()
    
#     x_train, y_train = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid = valid_df[feature_cols].values, valid_df[target_cols].values
    
# #     train_dataset = MoADataset(x_train, y_train)
# #     valid_dataset = MoADataset(x_valid, y_valid)
    
# #     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# #     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = TabNetRegressor(**tabnet_params)
    
#     model.fit(
#         X_train = X_train,
#         y_train = y_train,
#         eval_set = [(x_valid, y_valid)],
#         eval_name = ["val"],
#         eval_metric = ["logits_ll"],
#         max_epochs = MAX_EPOCH,
#         patience = 20,
#         batch_size = 1024, 
#         virtual_batch_size = 32,
#         num_workers = 1,
#         drop_last = False,
#         # To use binary cross entropy because this is not a regression problem
#         loss_fn = SmoothCrossEntropyLoss(smoothing=smoothing)
#     )
    

# #     model.to(DEVICE)
    
#     torch.backends.cudnn.benchmark = True
    
# #     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# #     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.3, div_factor=1e3, 
# #                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader)) 
    
# #     loss_fn = lambda inputs, targets : F.binary_cross_entropy((torch.clamp(torch.sigmoid(inputs), p_min, p_max)), targets)
# #     loss_tr = SmoothCrossEntropyLoss(smoothing=smoothing)
    
# #     early_stoppping_steps = EARLY_STOPPING_STEPS
# #     early_step = 0
    
#     oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
# #     best_loss = np.inf

#     ### Predict on validation ###
#     preds_val = model.predict(X_val)
    
# #     for epoch in range(EPOCHS):
        
# #         print(f'-----EPOCH{epoch+1}-----')
        
# #         train_loss, train_metric = train_fn(model, optimizer, scheduler, loss_tr, loss_fn, trainloader, DEVICE)
# #         print(f'train_loss: {train_loss:.5f}, train_metric: {train_metric:.5f}')
# #         train_losses.append(train_loss)
# #         valid_loss, valid_metric, valid_preds = valid_fn(model, loss_tr, loss_fn, validloader, DEVICE)
# #         print(f'valid_loss: {valid_loss:.5f}, valid_metric: {valid_metric:.5f}')
# #         valid_losses.append(valid_loss)
        
# #         if valid_loss < best_loss:
            
# #             best_loss = valid_loss
# #             oof[val_idx] = valid_preds
# #             torch.save(model.state_dict(), f'Simple_FOLD{fold+1}_SEED{seed}.pth')
            
# #         elif(EARLY_STOP == True):
            
# #             early_step += 1
# #             if (early_step >= early_stopping_steps):
# #                 break
                
# #     plt.plot(train_losses, label='train_losses')
# #     plt.plot(valid_losses, label='valid_losses')
# #     plt.xlabel('epochs')
# #     plt.ylabel('loss')
# #     plt.ylim([1e-2, 2e-2])
# #     plt.title(f'fold{fold+1} losses')
# #     plt.show()
                
#     #-----------------------PREDICTION-------------------------
                
#     x_test = test_[feature_cols].values
#     testdataset = TestDataset(x_test)
#     testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model_Simple(num_features=num_features, 
#                              num_targets=num_targets, 
#                              hidden_size_1=hidden_size_1,
#                              hidden_size_2=hidden_size_2)
    
#     model.load_state_dict(torch.load(f'Simple_FOLD{fold+1}_SEED{seed}.pth'))
#     model.to(DEVICE)
    
#     predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
#     predictions = inference_fn(model, testloader, DEVICE)
    
#     return oof, predictions

In [39]:
# from pytorch_tabnet.pretraining import TabNetPretrainer

# train_df = pd.get_dummies(train, columns=['cp_time', 'cp_dose'])
# test_df = pd.get_dummies(test, columns=['cp_time', 'cp_dose'])

# x_train = train_df[feature_cols].values
# x_test = test_df[feature_cols].values

# pretrainer = TabNetPretrainer(**tabnet_params)

# pretrainer.fit(X_train = x_test, 
#                eval_set = [x_train], 
#                max_epochs = MAX_EPOCH, 
#                patience = 20, 
#                batch_size = 1024, 
#                virtual_batch_size = 128, 
#                num_workers = 1, 
#                drop_last = False, 
#                pretraining_ratio = 0.8)

In [40]:
scores_auc_all = []
test_cv_preds = []

# mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

oof_preds = []
oof_targets = []
scores = []
scores_auc = []

oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))

SEED = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for seed in SEED:
    
    print(f'SEED: {seed}')
    
    oof_tmp = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
#     predictions_tmp = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    
    for fold in range(NFOLDS):
        print("FOLDS: ", fold + 1)
        print('*' * 60)

    #     X_train, y_train = train_df[feature_cols].values[train_idx, :], target[target_cols].values[train_idx, :]
    #     X_val, y_val = train_df[feature_cols].values[val_idx, :], target[target_cols].values[val_idx, :]
    #     X_test = test_df[feature_cols].values

        train_ = pd.get_dummies(folds, columns=['cp_time', 'cp_dose'])
        test_ = pd.get_dummies(test, columns=['cp_time', 'cp_dose'])

        trn_idx = train_[train_['kfold'] != fold].index
        val_idx = train_[train_['kfold'] == fold].index

        train_df = train_[train_['kfold'] != fold].reset_index()
        valid_df = train_[train_['kfold'] == fold].reset_index()
    
        x_train, y_train = train_df[feature_cols].values, train_df[target_cols].values
        x_valid, y_valid = valid_df[feature_cols].values, valid_df[target_cols].values

        x_test = test_[feature_cols].values

        ### Model ###
        model = TabNetRegressor(**tabnet_params)

        ### Fit ###
        # Another change to the original code
        # virtual_batch_size of 32 instead of 128
        model.fit(
            X_train = x_train,
            y_train = y_train,
            eval_set = [(x_valid, y_valid)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs = MAX_EPOCH,
            patience = 20,
            batch_size = 1024, 
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
#             from_unsupervised=pretrainer,
            # To use binary cross entropy because this is not a regression problem
            loss_fn = SmoothBCEwLogits(smoothing=smoothing)
        )
        print('-' * 60)

        ### Predict on validation ###
        preds_val = model.predict(x_valid)
        # Apply sigmoid to the predictions
        preds = 1 / (1 + np.exp(-preds_val))
        preds = np.clip(preds, p_min, p_max)
        oof_tmp[val_idx] += preds
        score = np.min(model.history["val_logits_ll"])

        ### Save OOF for CV ###
        oof_preds.append(preds)
        oof_targets.append(y_valid)
        scores.append(score)

        ### Predict on test ###
        preds_test = model.predict(x_test)
        preds_test = 1 / (1 + np.exp(-preds_test))
        preds_test = np.clip(preds_test, p_min, p_max)
#         predictions_tmp += preds_test / NFOLDS
        test_cv_preds.append(preds_test)

        name = f'TabNet_FOLD{fold+1}_SEED{seed}'
        model.save_model(name)
    
    oof += oof_tmp / len(SEED)

train[target_cols] = oof
test_preds_all = np.stack(test_cv_preds)

SEED: 0
FOLDS:  1
************************************************************
Device used : cuda
epoch 0  | loss: 0.35766 | val_logits_ll: 0.03927 |  0:00:02s
epoch 1  | loss: 0.02866 | val_logits_ll: 0.02675 |  0:00:04s
epoch 2  | loss: 0.02379 | val_logits_ll: 0.0219  |  0:00:06s
epoch 3  | loss: 0.02178 | val_logits_ll: 0.02115 |  0:00:07s
epoch 4  | loss: 0.02129 | val_logits_ll: 0.02083 |  0:00:09s
epoch 5  | loss: 0.02095 | val_logits_ll: 0.02055 |  0:00:11s
epoch 6  | loss: 0.02061 | val_logits_ll: 0.02039 |  0:00:13s
epoch 7  | loss: 0.02044 | val_logits_ll: 0.02019 |  0:00:14s
epoch 8  | loss: 0.02008 | val_logits_ll: 0.01984 |  0:00:16s
epoch 9  | loss: 0.01972 | val_logits_ll: 0.01936 |  0:00:18s
epoch 10 | loss: 0.01934 | val_logits_ll: 0.01918 |  0:00:19s
epoch 11 | loss: 0.01892 | val_logits_ll: 0.01878 |  0:00:21s
epoch 12 | loss: 0.01857 | val_logits_ll: 0.01905 |  0:00:23s
epoch 13 | loss: 0.01842 | val_logits_ll: 0.01816 |  0:00:25s
epoch 14 | loss: 0.01822 | val_log

In [41]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(
    train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

cv = 0
for i in range(len(target_cols)):
    cv_ = log_loss(y_true[:, i], y_pred[:, i])
    cv += cv_ / len(target_cols)
    
auc = 0
for i in range(len(target_cols)):
    auc_ = roc_auc_score(y_true[:, i], y_pred[:, i])
    auc += auc_ / len(target_cols)
    
print(f'CV log_loss: {cv:.6f}')
print(f'AUC: {auc:.6f}')

CV log_loss: 0.015295
AUC: 0.772304


In [42]:
file_path = 'oof_TabNet.npy'
np.save(file_path, y_pred)

all_feat = [col for col in sample_submission.columns if col not in ["sig_id"]]
# To obtain the same length of test_preds_all and submission
data_path = "../input/lish-moa/"
test = pd.read_csv(data_path + "test_features.csv")
sig_id = test[test["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)
tmp = pd.DataFrame(test_preds_all.mean(axis = 0), columns = all_feat)
tmp["sig_id"] = sig_id

submission = pd.merge(test[["sig_id"]], tmp, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)

#submission[all_feat] = tmp.mean(axis = 0)

# Set control to 0
#submission.loc[test["cp_type"] == 0, submission.columns[1:]] = 0
submission.to_csv("submission.csv", index = None)
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001047,0.001311,0.002143,0.015446,0.018633,0.004351,0.002886,0.003779,0.000329,...,0.000702,0.001104,0.004033,0.000708,0.000858,0.000586,0.000564,0.002206,0.003183,0.001678
1,id_001897cda,0.000689,0.001330,0.001827,0.002157,0.001336,0.001591,0.006605,0.007658,0.002771,...,0.000958,0.000731,0.005163,0.000150,0.005120,0.000689,0.009592,0.001189,0.003206,0.002114
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000859,0.001027,0.001338,0.006748,0.014360,0.003616,0.002771,0.005120,0.000472,...,0.000604,0.001303,0.001807,0.004065,0.008173,0.000621,0.003713,0.001664,0.001003,0.001718
4,id_0027f1083,0.001304,0.001590,0.001589,0.014424,0.022047,0.005089,0.003502,0.002617,0.000559,...,0.000693,0.000808,0.003033,0.001356,0.001447,0.000671,0.000748,0.002165,0.000451,0.001579


In [43]:
tmp

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,sig_id
0,0.001047,0.001311,0.002143,0.015446,0.018633,0.004351,0.002886,0.003779,0.000329,0.008192,...,0.001104,0.004033,0.000708,0.000858,0.000586,0.000564,0.002206,0.003183,0.001678,id_0004d9e33
1,0.000689,0.001330,0.001827,0.002157,0.001336,0.001591,0.006605,0.007658,0.002771,0.010145,...,0.000731,0.005163,0.000150,0.005120,0.000689,0.009592,0.001189,0.003206,0.002114,id_001897cda
2,0.000859,0.001027,0.001338,0.006748,0.014360,0.003616,0.002771,0.005120,0.000472,0.011087,...,0.001303,0.001807,0.004065,0.008173,0.000621,0.003713,0.001664,0.001003,0.001718,id_00276f245
3,0.001304,0.001590,0.001589,0.014424,0.022047,0.005089,0.003502,0.002617,0.000559,0.012938,...,0.000808,0.003033,0.001356,0.001447,0.000671,0.000748,0.002165,0.000451,0.001579,id_0027f1083
4,0.001112,0.001030,0.001821,0.024444,0.030086,0.005542,0.004745,0.002402,0.000485,0.013019,...,0.001036,0.003558,0.014641,0.001181,0.000535,0.000762,0.002339,0.000299,0.001449,id_006fc47b8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,0.000709,0.001173,0.001791,0.002886,0.005732,0.003120,0.001597,0.003319,0.000392,0.004469,...,0.003697,0.002329,0.143120,0.014281,0.000696,0.007439,0.001145,0.000562,0.000969,id_ff7004b87
3620,0.001656,0.001775,0.001474,0.015352,0.025165,0.005868,0.005821,0.004174,0.000697,0.017693,...,0.001049,0.003420,0.002869,0.001700,0.000726,0.000557,0.001996,0.000409,0.001965,id_ff925dd0d
3621,0.001516,0.001264,0.001289,0.013874,0.044084,0.006841,0.002423,0.007861,0.000234,0.013064,...,0.000529,0.002336,0.001945,0.001983,0.000617,0.002448,0.001881,0.000886,0.001248,id_ffb710450
3622,0.001395,0.001221,0.001475,0.022291,0.023385,0.004559,0.010480,0.002801,0.001428,0.036956,...,0.000888,0.002833,0.001936,0.001406,0.000507,0.001113,0.002120,0.000279,0.002516,id_ffbb869f2
